In [24]:
# Analyse tournament results
import os 
import sys
import pandas as pd
import requests
import glob
import json

# Build the path to the pdb files
file_dir = os.getcwd()
folder_name = file_dir.split('/')[-1]
repo_dir = file_dir.replace(f'/{folder_name}', '')
plot_dir = f'{file_dir}/plots'
pdb_dir = f"{repo_dir}/tournament_benchmark/gpcr_tournament"
principal_agonists_df = pd.read_csv(f"{repo_dir}/classifier_benchmark_data/output/6_interactions_with_decoys.csv")
principal_agonists_df = principal_agonists_df[principal_agonists_df["Decoy Type"] == "Principal Agonist"]

# Get the pdb files - list all pdb files in the pdb_dir directory and its subdirectories
pdb_files = []
for root, dirs, files in os.walk(pdb_dir):
    for file in files:
        if file.endswith(".pdb"):
            pdb_files.append(os.path.join(root, file))

# Use glob to find all matching files
pattern = os.path.join(repo_dir, "tournament_benchmark/gpcr_tournament/*/msas/chain_id_map.json")
matching_files = glob.glob(pattern)

# Print the list of matching files
for file_path in matching_files:

    # Read the chain_id_map.json file
    chain_id_map = json.load(open(file_path))
    receptor = chain_id_map["A"]["description"]
    principal_agonist = principal_agonists_df[principal_agonists_df["Target ID"] == receptor]["Decoy ID"].values[0]

    ligand_chain = "None"
    for key in chain_id_map:
        if str(chain_id_map[key]["description"]) == str(principal_agonist):
            ligand_chain = key
    
    # Set the column value in principal_agonists_df to the ligand chain
    principal_agonists_df.loc[principal_agonists_df["Target ID"] == receptor, "Ligand Chain"] = ligand_chain

In [30]:
# Read in the chosen GRNs
grn_path = f"{repo_dir}/grn_analysis/chosen_grns.txt"
with open(grn_path) as f:
    chosen_grns = f.readlines()
    chosen_grns = [grn.strip() for grn in chosen_grns]

['7x34', '7x38', '45x52', '3x32', '45x51', '6x55', '1x32', '4x61']


In [38]:
'''
Script to check whether a predicted receptor-ligand complex includes interactions between the ligand and the receptor.
The interactions will only focus on the generic residue numbers that are observed in the dataset of stucturally resolved complexes of 
GPCRs and endogenous peptide ligands or their non-endogenous analogues.
'''

import subprocess 
import os
from Bio.PDB.NeighborSearch import NeighborSearch
import pandas as pd
import glob
import csv 
import json

def get_chains_from_pdb(pdb_path, chain_to_get, output_path):
    """
    Helper function to get specific chains from a pdb file
    Note: requires pdb-tools to be installed using: pip install pdb-tools
    """
    chain_to_get = ",".join(chain_to_get)

    # Command to run on terminal - uses pdb-tools pdb_selchain to extract receptor and peptide chain from PDB file
    command = f"pdb_selchain -{chain_to_get} {pdb_path} > {output_path}"

    # Run command on terminal
    os.system(command)

    return output_path

def apply_generic_numbering(pdb_path, output_path, chain_to_get="B"):
    '''
    Function to apply generic numbering to a PDB file of a receptor. 
    '''
    # Apply generic numbering to the pdb file
    command = f'curl -X POST -F "pdb_file=@{pdb_path}" https://gpcrdb.org/services/structure/assign_generic_numbers'
    output = subprocess.check_output(command, shell=True)
    # Make sure output directory exists
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with open(output_path, 'wb') as f:
        f.write(output)

    # Compare lines in the input pdb and the output pdb 
    # and empty b-factor field in the output pdb so that only GRN
    # is stored in the b-factor field
    with open(pdb_path, 'r') as f:
        pdb = f.readlines()
        pdb = [line.strip() for line in pdb]
        pdb = [line for line in pdb if line.startswith("ATOM")]
        pdb = [line for line in pdb if line[21] == chain_to_get]
        
    with open(output_path, 'r') as f:
        output_pdb = f.readlines()
        output_pdb = [line.strip() for line in output_pdb]     
        output_pdb = [line for line in output_pdb if line.startswith("ATOM")] 
        output_pdb = [line for line in output_pdb if line[21] == chain_to_get]

    # Make a dictionary from residue number to generic residue number
    grn_dict = {}
    for i in range(len(pdb)):
        if pdb[i] != output_pdb[i]:
            residue_number = pdb[i][22:26].strip()
            generic_number = output_pdb[i][60:66].strip()
            grn_dict[residue_number] = generic_number
        else:
            # Replace the b-factor field with empty space
            output_pdb[i] = output_pdb[i][:60] + "     0" + output_pdb[i][66:]

    # Save the dictionary into a json file
    grn_json_path = output_path.replace(".pdb", ".json")
    with open(grn_json_path, 'w') as f:
        json.dump(grn_dict, f)

    # Save the pdb file with generic residue numbers
    with open(output_path, 'w') as f:
        for line in output_pdb:
            f.write(line + "\n")

    return output_path

def get_grn_positions(grn_pdb_path, receptor_chain):

    # Open grn_pdb file
    with open(grn_pdb_path, 'r') as f:
        grn_pdb = f.readlines()
        grn_pdb = [line.strip() for line in grn_pdb]

    # Get only CA atoms from the receptor chain
    ca_grn_pdb = [line for line in grn_pdb if "CA" == line[12:16].strip()]
    ca_grn_pdb = [line for line in ca_grn_pdb if line[21] == receptor_chain]

    # Make a dictionary with residue numbers as keys and generic residue numbers as values
    grn_dict = {}
    for grn_position in ca_grn_pdb:
        residue_number = grn_position[22:26].strip()
        generic_number = grn_position[60:66].strip()
        grn_dict[residue_number] = generic_number

    return grn_dict

def get_atom_list(pdb_path, grn_folder, receptor_chain = "B", ligand_chain = "C", search_radius = 6.6):

    # Open pdb using biopython
    from Bio.PDB import PDBParser
    parser = PDBParser()
    pdb_name = pdb_path.split("/")[-1].split(".")[0]
    structure = parser.get_structure(pdb_name, pdb_path)

    # Save atom objects into two lists: one for the receptor and one for the peptide
    atom_list_receptor = []
    atom_list_peptide = []
    for atom in structure.get_atoms():
        chain_id = atom.get_full_id()[2]
        if chain_id == receptor_chain:
            atom_list_receptor.append(atom)
        elif chain_id == ligand_chain:
            atom_list_peptide.append(atom)

    # Apply generic numbering to the receptor
    receptor_name = pdb_name.split("___")[0]
    grn_path = os.path.join(grn_folder, receptor_name + "_grn.pdb")
    grn_json_path = grn_path.replace(".pdb", ".json")
    if not os.path.exists(grn_path):
        apply_generic_numbering(pdb_path, grn_path)

    # Load the generic residue numbers from the json file
    with open(grn_json_path, 'r') as f:
        generic_residue_numbers = json.load(f)
     
    # Filter receptor atom list based on generic residue numbers
    grn_list_receptor = []
    for atom in atom_list_receptor:
        # Get residue number of atom
        residue_number = str(atom.get_parent().get_id()[1])
        if residue_number in generic_residue_numbers:
            grn_list_receptor.append(atom)

    # Concatenate two lists
    grn_list_receptor.extend(atom_list_peptide)

    # Get interactions between receptor and peptide
    ns = NeighborSearch(grn_list_receptor)
    all_neighbors = ns.search_all(search_radius, "R")
    all_neighbors = [pair for pair in all_neighbors if pair[0].get_full_id()[2] != pair[1].get_full_id()[2]]
    all_neighbors = sorted(all_neighbors, key=lambda x: x[0].get_full_id())
    
    # Return generic residue numbers for interacting receptor positions
    grns = []
    for pair in all_neighbors:
        res_n = pair[0].get_full_id()[-1][1]
        grns.append(generic_residue_numbers[str(res_n)])
    grns = list(set(grns))
    return sorted(grns)
    
# Path to the folder that contains the generic residue numbers for each receptor
grn_folder = f"{repo_dir}/tournament_benchmark/grn_pdbs"

# List all pdbs in pdb_directory and its subdirectories
pdb_directory = f"{repo_dir}/tournament_benchmark/gpcr_tournament"

output_csv = f"{repo_dir}/tournament_benchmark/interacting_grns.csv"

# Process each PDB file and write results to the CSV file line by line
with open(output_csv, "w", newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["pdb", "interacting_grns"])  # Write header
    c = 0
    for pdb in pdb_files:
        c+=1
        if os.path.exists(pdb):
            print(f"Processing {pdb}, {c}/{len(pdb_files)}", flush = True)
            grns = get_atom_list(pdb, grn_folder) 
            pdb_name = os.path.basename(pdb).split(".")[0]
            print(f"{pdb} - GRNs: {grns}")
            csv_writer.writerow([pdb_name, grns])


Processing /Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/ackr2_tournament/ackr2_tournament.pdb, 1/124


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  410k    0     0  100  410k      0  71245  0:00:05  0:00:05 --:--:-- 55300

/Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/ackr2_tournament/ackr2_tournament.pdb - GRNs: ['1.24', '1.25', '1.27', '1.28', '1.35', '1.39', '2.57', '2.60', '2.61', '2.64', '3.28', '3.29', '3.30', '3.32', '3.33', '3.36', '3.37', '4.56', '4.57', '4.61', '4.65', '45.50', '45.51', '45.52', '5.32', '5.33', '5.36', '5.39', '5.40', '5.43', '6.51', '6.55', '6.58', '6.61', '6.62', '6.63', '6.64', '7.23', '7.24', '7.25', '7.27', '7.28', '7.30', '7.31', '7.32', '7.34', '7.35', '7.38', '7.42']
Processing /Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/qrfpr_tournament/qrfpr_tournament.pdb, 2/124


100  821k  100  410k  100  410k  68396  68470  0:00:06  0:00:06 --:--:--  113k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  477k    0     0  100  477k      0  86789  0:00:05  0:00:05 --:--:-- 67847

/Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/qrfpr_tournament/qrfpr_tournament.pdb - GRNs: ['-2.55', '-5.46', '2.56', '2.57', '2.59', '2.60', '2.61', '2.62', '2.63', '2.64', '2.65', '2.66', '2.67', '23.49', '23.50', '23.51', '3.24', '3.25', '3.26', '3.28', '3.29', '3.32', '3.33', '3.34', '3.36', '3.37', '4.60', '4.63', '4.64', '45.50', '45.51', '45.52', '5.35', '5.36', '5.37', '5.39', '5.40', '5.41', '5.43', '6.48', '6.51', '6.52', '6.55', '6.59', '7.31', '7.34', '7.38', '7.42']
Processing /Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/npy4r_tournament/npy4r_tournament.pdb, 3/124


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  410k    0     0  100  410k      0  89393  0:00:04  0:00:04 --:--:-- 89387

/Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/ccr2_tournament/ccr2_tournament.pdb - GRNs: ['1.27', '1.28', '1.31', '1.32', '1.35', '1.39', '2.56', '2.57', '2.60', '2.63', '2.64', '2.65', '2.66', '23.49', '3.25', '3.28', '3.29', '3.30', '3.32', '3.33', '4.61', '4.64', '4.65', '45.50', '45.51', '45.52', '5.32', '5.33', '5.36', '5.39', '5.43', '6.51', '6.55', '6.58', '6.59', '6.61', '6.62', '7.25', '7.26', '7.27', '7.28', '7.31', '7.32', '7.34', '7.35', '7.36', '7.38', '7.39', '7.42']
Processing /Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/cxcr5_tournament/cxcr5_tournament.pdb, 124/124


100  820k  100  409k  100  410k  84324  84416  0:00:04  0:00:04 --:--:--  114k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  665k    0     0  100  665k      0   102k  0:00:06  0:00:06 --:--:--     0

/Users/pqh443/Documents/Git_projects/GPRC_peptide_benchmarking/tournament_benchmark/gpcr_tournament/cxcr5_tournament/cxcr5_tournament.pdb - GRNs: ['1.31', '1.35', '1.36', '1.39', '2.53', '2.56', '2.57', '2.59', '2.60', '2.61', '2.62', '2.63', '2.64', '2.65', '23.50', '3.24', '3.25', '3.28', '3.29', '3.32', '3.33', '45.50', '45.51', '45.52', '5.36', '5.37', '5.40', '6.51', '6.54', '6.55', '6.58', '6.61', '6.62', '6.63', '6.64', '7.23', '7.24', '7.26', '7.27', '7.28', '7.29', '7.30', '7.31', '7.32', '7.34', '7.35', '7.36', '7.38', '7.42']


100 1331k  100  665k  100  665k  91189  91251  0:00:07  0:00:07 --:--:--  167k
